# Tutorial on manipulating OLCI L1 EFR product from the cloud

In [ ]:
import xarray as xr
from pathlib import Path
from xarray_eop.eop import open_eop_datatree

### Get AWS credentials

In [ ]:
import s3fs
import json
# Assuming credentials are in ~/.eopf/secrets.json
try:
    SECRET_PATH = Path.home() / Path(".eopf/secrets.json")
    with open(SECRET_PATH) as f:
        secrets=json.load(f)

# Or use ENV variables
except:
    import os
    secrets = {"s3input" : {
        "key": os.getenv("AWS_ACCESS_KEY_ID"),
        "secret": os.getenv("AWS_SECRET_ACCES_KEY"),
        "endpoint_url": os.getenv("AWS_ENDPOINT_URL"),
        "region_name": os.getenv("AWS_DEFAULT_REGION")
        }
    }

secrets["s3input"].pop("region_name",None)


### Browse S3 buckets and get OLCEFR product

In [ ]:
import s3fs
SAMPLE_PATH = "s3://s3-input/Products/NewFormat/"
s3 = s3fs.S3FileSystem(
    key=secrets["s3input"]["key"],
    secret=secrets["s3input"]["secret"],
    endpoint_url=secrets["s3input"]["endpoint_url"]
)
s3_path = s3.glob(SAMPLE_PATH+"S3OLCEFR*.zip")
store=f"zip::s3://{s3_path[0]}"
store

### Opening the whole product datatree

In [ ]:
dt = open_eop_datatree(store,backend_kwargs={"storage_options": {"s3":secrets["s3input"]}})
dt

### Opening measurement radiances

In [ ]:
rad = dt.measurements.image.oa01_radiance
rad

Underlying data is dask.array

In [ ]:
rad.data

#### Simple raster plot
Note that using xarray, data is correctly decoded and masked

In [ ]:
rad.plot()

#### Plot using the coordinates (lon,lat)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14, 6))
ax = plt.axes()
rad.plot.pcolormesh(
    ax=ax,
    x="longitude", y="latitude", add_colorbar=False
)

#### Open Meteorological conditions

In [ ]:
meteo=dt.conditions.meteo
meteo

#### Interpolate the atmo. temp. profile at p=832.2 hPa and plot

In [ ]:
tp=meteo["atmospheric_temperature_profile"].interp(tie_pressure_levels=832.2)
tp.plot()

#### Open sat/sun angles
Angles are stored on a tiepoint subgrid

In [ ]:
geometry=dt.conditions.geometry

#### Have a look at condition parameters for removed pixels

In [ ]:
dt.conditions.orphans